# MAE 598: Design Optimization - Homework 3

## Problem 1

## Problem 2

## Problem 3

## Problem 4

In [75]:
# Import libraries
import torch as t
import numpy as np
import torch.nn as nn
from torch.autograd import Variable

from torch.autograd.functional import jacobian
from matplotlib import pyplot as plt


In [89]:
# Part 1
"""
Decision variable: x0
State variables: x1, x2
"""
m = 2
n = 3

def f(x):
    f = lambda x: x[0] ** 2 + x[1] ** 2 + x[2] ** 2
    # Define the constraints
    h1 = lambda x: ((x[0] ** 2) / 4) + ((x[1] ** 2) / 5) + ((x[2] ** 2) / 25) - 1
    h2 = lambda x: x[0] + x[1] - x[2]
    
    return f(x), h1(x), h2(x)

# Part 2
#Defining input tensors
# x = Variable(t.tensor([1., 1., 1.], dtype=t.float64), requires_grad=True)
x = t.tensor([1., 1., 1.], dtype=t.float, requires_grad=True)

# Reduced Gradient (Analytically computed)
dfdd_analytical = lambda x: ((-5 * x[0] * x[2]) + (-21 * x[0] * x[2]) + (16 * x[1] * x[2])) / (10 * x[1] + 2 * x[2])

In [90]:
print('Functions evaluated at the initial gues:\n ', f(x))
print('\nJacobian evaluated at the initial guess:\n', jacobian(f, (x)))

Functions evaluated at the initial gues:
  (tensor(3., grad_fn=<AddBackward0>), tensor(-0.5100, grad_fn=<SubBackward0>), tensor(1., grad_fn=<SubBackward0>))

Jacobian evaluated at the initial guess:
 (tensor([2., 2., 2.]), tensor([0.5000, 0.4000, 0.0800]), tensor([ 1.,  1., -1.]))


In [91]:
J = t.zeros((len(j), len(j)))
for p in range(len(j)):
   J[p] =  jacobian(f, (x))[p]
print('Jacobian:\n', J)

Jacobian:
 tensor([[ 2.0000,  2.0000,  2.0000],
        [ 0.5000,  0.4000,  0.0800],
        [ 1.0000,  1.0000, -1.0000]])


In [92]:
# Part 3
# Reduced gradient
dfdd = J[0,0] - J[0,1:] @ np.linalg.inv(J[1:,1:]) @ J[1:,0]

print('Reduced gradient:', dfdd)
print('Analytical calculation', dfdd_analytical(x))

Reduced gradient: tensor(-0.8333)
Analytical calculation tensor(-0.8333, grad_fn=<DivBackward0>)


In [98]:
def LMSolve(f, x, m=m, n=n, epochs=200, Lambda=10.):
    H = t.zeros((m,1))
    J = t.zeros((n, n))
    # Lambda = 50

    for q in range(epochs):
        for r in range(m):
           H[r] =  f(x)[1:][r]
        # print('\nH: ', H)

        for p in range(n):
           J[p] =  jacobian(f, (x))[p]
        # print('Jacobian:\n', J)


        delta = t.inverse(J[1:,1:].T @ J[1:,1:] + Lambda * t.eye(2)) @ J[1:,1:].T @ H
        # print(delta.shape)

        with t.no_grad():
            x[1:] = x[1:] - delta.T

        if q % 20 == 19:
            print(f'\nEpoch [{q}]: [dk, sk] --> {x}')
            
    return x
        

In [99]:
# Find the feasible point based on the initial gues of decision variables
x = LMSolve(f, x)
print('\n Point x: ', x)


Epoch [19]: [dk sk] --> tensor([1.0000, 1.5614, 2.5614], requires_grad=True)

Epoch [39]: [dk sk] --> tensor([1.0000, 1.5614, 2.5614], requires_grad=True)

Epoch [59]: [dk sk] --> tensor([1.0000, 1.5614, 2.5614], requires_grad=True)

Epoch [79]: [dk sk] --> tensor([1.0000, 1.5614, 2.5614], requires_grad=True)

Epoch [99]: [dk sk] --> tensor([1.0000, 1.5614, 2.5614], requires_grad=True)

Epoch [119]: [dk sk] --> tensor([1.0000, 1.5614, 2.5614], requires_grad=True)

Epoch [139]: [dk sk] --> tensor([1.0000, 1.5614, 2.5614], requires_grad=True)

Epoch [159]: [dk sk] --> tensor([1.0000, 1.5614, 2.5614], requires_grad=True)

Epoch [179]: [dk sk] --> tensor([1.0000, 1.5614, 2.5614], requires_grad=True)

Epoch [199]: [dk sk] --> tensor([1.0000, 1.5614, 2.5614], requires_grad=True)

 Point x:  tensor([1.0000, 1.5614, 2.5614], requires_grad=True)


In [ ]:
# Part 4
# Initialization
t0 = 0.5
e = np.linalg.norm(dfdd)
tol = 1e-4 # Error threshold
K = 50 # Max no. of iterations
# xSol = x.T
# fVal = [f(x)[0]]
# alphaSol = [1]

k = 0
while e > tol:

    alpha = 1
    i = 0
    
    dk0 = x[0] - alpha * dfdd
    sk0 = x[1:] + t.from_numpy(alpha * np.linalg.inv(J[1:,1:]) @ J[1:,0] * J[0,0])
    func = f(t.hstack((dk0, sk0)))[0].item()
    phi = f(x)[0].item() - (t0 * alpha * dfdd ** 2) 
    
    # Part 4.1
    # Inexact line search
    while func > phi and i < K:
        alpha = 0.5 * alpha
        dk0 = x[0] - alpha * dfdd
        sk0 = x[1:] + t.from_numpy(alpha * np.linalg.inv(J[1:,1:]) @ J[1:,0] * J[0,0])
        func = f(t.hstack((dk0, sk0)))[0].item()
        phi = f(x)[0].item() - (t0 * alpha * dfdd ** 2) 
        print(f'Iteration: {i}, alpha: {alpha}')
        i += 1


    # Part 4.2
    # Tensor
    dk0 = x[0] - alpha * dfdd
    # Part 4.3
    sk0 = x[1:] + t.from_numpy(alpha * np.linalg.inv(J[1:,1:]) @ J[1:,0] * J[0,0])
    
    
    # Part 4.4
    # LM Solver
    Lambda = 10
    for q in range(10):
        h = f(t.hstack((dk0, sk0)))[1:,:].item()
        j = jacobian(f, (t.hstack((dk0, sk0))))
        J = np.zeros((len(j), len(j)))
        for p in range(len(j)):
           J[p] =  j[p].detach().numpy()
        print(J)
        
        delta = np.linalg.inv(J[1:,1:].T @ J[1:,1:] + Lambda * np.eye(3)) @ J[1:,1:].T @ h
    
        sk0 = sk0 - delta

    
    
    # Update x
    x[0] = dk0
    x[1:] = sk0
    
    # Part 4.5
    
    
    j = jacobian(f, (x))
    J = np.zeros((len(j), len(j)))
    for p in range(len(j)):
       J[p] =  j[p].detach().numpy()
    print(J)
    
    dfdd = J[0,0] - J[0,1:] @ np.linalg.inv(J[1:,1:]) @ J[1:,0]
    e = np.linalg.norm(dfdd)
    
    
    # xSol = np.concatenate((xSol, x.T), axis=0)  # Record x values in each iteration
    # fVal.append(f(x)[0]) # Record f values in each iteration
    # alphaSol.append(alpha) # Record alpha values in each iteration

    k += 1
    print (f"Iteration: {k:<5} Alpha: {alpha:<10} x: {str(x.T[0, :]) :<30} f(x): {fVal[k]:<30}")








TypeError: 'NoneType' object is not subscriptable

In [ ]:
# print("\nGRG\n")

# # Initial guess   
# x1 = np.array([[0], [0]])
# print('\nSolution #1\n')
# xSol1, fVal1, alphaSol1 = newt(x1)
# # Convergence analysis
# yplt1 = abs(np.array(fVal1) - fStar * np.ones([1, len(fVal1)])[0])
# # Plot results
# print("\nConvergence plot\n")
# plt.semilogy(range(1, len(fVal1)+1), yplt1, "rs-")
# plt.xlim(1, 50)
# plt.xticks(ticks=range(1, len(fVal1)+1), labels=range(1, len(fVal1)+1))
# plt.xlabel("Iterations (k)")
# plt.ylabel("log|f_k - f*|")
# plt.title("Convergence analysis 1")
# plt.show()


## Problem 5